In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pathlib import Path

part_path = Path("part-2")
part_path.mkdir(exist_ok=True)

raw_path = Path(f"{part_path}/raw")
raw_path.mkdir(exist_ok=True)

processed_path = Path(f"{part_path}/processed")
processed_path.mkdir(exist_ok=True)

submission_path = Path(f"{part_path}/submission")
submission_path.mkdir(exist_ok=True)

In [3]:
def stringify_array(arr):
    return "'" + "','".join(arr) + "'"

In [4]:
def remove_str(projects):
    projects = [p.replace('https://github.com/', '') for p in projects]
    return projects

In [5]:
def remove_str_df(df):
    df['project_a'] = df['project_a'].str.replace('https://github.com/', '')
    df['project_b'] = df['project_b'].str.replace('https://github.com/', '')
    return df


In [6]:
def get_dataset():
    if str(part_path) == 'part-1':
        # repository_url = (
        #     "https://raw.githubusercontent.com/deepfunding/mini-contest/refs/heads/main/"
        # )
        
        # df_train = pd.read_csv(f"{repository_url}/dataset.csv")
        # df_test = pd.read_csv(f"{repository_url}/test.csv")
        # df_train.to_csv(f"{part_path}/train.csv", index=False)
        # df_test.to_csv(f"{part_path}/test.csv", index=False)
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
        
    elif str(part_path) == 'part-2':
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
     
    return df_train, df_test

In [7]:
df_train, df_test = get_dataset()

In [9]:
projects = pd.concat([
    df_train['project_a'],
    df_train['project_b'],
    df_test['project_a'],
    df_test['project_b']
]).unique().tolist()

In [10]:
projects[:5]

['https://github.com/mochajs/mocha',
 'https://github.com/chzyer/readline',
 'https://github.com/gulpjs/gulp',
 'https://github.com/webpack/webpack',
 'https://github.com/redux-saga/redux-saga']

In [11]:
len(projects)

117

In [12]:
trimed_projects = remove_str(projects)

In [13]:
df_train.quarter.unique()

array(['2016-04', '2016-07', '2016-10', '2017-01', '2017-04', '2017-07',
       '2017-10', '2018-01', '2018-04', '2018-07', '2018-10', '2019-01',
       '2019-04', '2019-07', '2019-10', '2020-01', '2020-04', '2020-07',
       '2020-10', '2021-01', '2021-04', '2021-07', '2021-10', '2022-01',
       '2022-04', '2022-07', '2022-10', '2023-01', '2023-04', '2023-07',
       '2023-10'], dtype=object)

In [14]:
df_test.quarter.unique()

array(['2024-01', '2024-04', '2024-07', '2024-10'], dtype=object)

In [63]:
df_train = pd.read_csv(f"{processed_path}/train-pre-embeddings.csv")
df_test = pd.read_csv(f"{processed_path}/test-pre-embeddings.csv")

In [50]:
import numpy as np
# Create quarter to index mapping
quarters = df_train.quarter.unique().tolist() + df_test.quarter.unique().tolist()
quarters.sort()  # Sort chronologically
quarter_to_index = {quarter: idx for idx, quarter in enumerate(quarters, 1)}

# Calculate weights as before
indices = np.arange(1, 36)
weights = 0.5 + ((indices - 1) / 34) * 0.5
weights_list = weights.tolist()

# Create quarter to weight mapping
quarter_to_weight = {quarter: weights_list[idx-1] for quarter, idx in quarter_to_index.items()}

# Example usage:
print("Quarter mappings:")
for quarter, weight in quarter_to_weight.items():
    print(f"{quarter}: {weight:.4f}")

Quarter mappings:
2016-04: 0.5000
2016-07: 0.5147
2016-10: 0.5294
2017-01: 0.5441
2017-04: 0.5588
2017-07: 0.5735
2017-10: 0.5882
2018-01: 0.6029
2018-04: 0.6176
2018-07: 0.6324
2018-10: 0.6471
2019-01: 0.6618
2019-04: 0.6765
2019-07: 0.6912
2019-10: 0.7059
2020-01: 0.7206
2020-04: 0.7353
2020-07: 0.7500
2020-10: 0.7647
2021-01: 0.7794
2021-04: 0.7941
2021-07: 0.8088
2021-10: 0.8235
2022-01: 0.8382
2022-04: 0.8529
2022-07: 0.8676
2022-10: 0.8824
2023-01: 0.8971
2023-04: 0.9118
2023-07: 0.9265
2023-10: 0.9412
2024-01: 0.9559
2024-04: 0.9706
2024-07: 0.9853
2024-10: 1.0000


In [64]:
base_features = [
   "size", 
   "size_b", 
   "size_ratio",
   "stars", 
   "stars_b", 
   "stars_ratio",
   "watchers",
   "watchers_b",
   "watchers_ratio",
   "forks", 
   "forks_b", 
   "forks_ratio", 
   "open_issues", 
   "open_issues_b", 
   "issues_ratio",
   "subscribers_count", 
   "subscribers_count_b",  
   "subscribers_ratio",
   "commit_code",
   "commit_code_b",
   "commits_ratio",
   "forked",
   "forked_b",
   "forked_ratio",
   "issue_closed",
   "issue_closed_b",
   "issue_closed_ratio",
   "issue_comment",
   "issue_comment_b",
   "issue_comment_ratio",
   "issue_opened",
   "issue_opened_b",
   "issue_opened_ratio",
   "issue_reopened",
   "issue_reopened_b",
   "issue_reopened_ratio",
   "pull_request_closed",
   "pull_request_closed_b",
   "pull_request_closed_ratio",
   "pull_request_merged",
   "pull_request_merged_b",
   "pull_request_merged_ratio",
   "pull_request_opened",
   "pull_request_opened_b",
   "pull_request_opened_ratio",
   "pull_request_reopened",
   "pull_request_reopened_b",
   "pull_request_reopened_ratio",
   "pull_request_review_comment",
   "pull_request_review_comment_b",
   "pull_request_review_comment_ratio",
   "release_published",
   "release_published_b",
   "release_published_ratio",
   "starred",
   "starred_b",
   "starred_ratio",
   "v_index",
   "v_index_b",
   "v_index_ratio",
   "stars_intersection_v_index",
   "stars_b_intersection_v_index_b",
   "stars_ratio_intersection_v_index_ratio",
   "num_dependents",
   "num_dependents_b",
   "dependency_rank",
   "dependency_rank_b",
   "num_dependents_ratio",
]

In [65]:
# Get the weights for each row based on quarter
weights = df_train['quarter'].map(quarter_to_weight)

# Apply weights to features
for base in base_features:
    # Weight features ending in _a
    df_train[f'{base}'] = df_train[f'{base}'] * weights
    
# Do the same for test data
weights_test = df_test['quarter'].map(quarter_to_weight)
for base in base_features:
    df_test[f'{base}'] = df_test[f'{base}'] * weights_test
    
# Display example to verify
print("Sample of weighted features:")
sample_cols = ['quarter', 'stars', 'stars_b']
print(df_train[sample_cols].head())

Sample of weighted features:
   quarter         stars       stars_b
0  2016-04  11349.500000   1054.500000
1  2016-04  11349.500000  16531.000000
2  2016-04   1054.500000  16531.000000
3  2016-07   1085.514706  17017.205882
4  2016-07   1085.514706  11683.308824


In [68]:
df_train.to_csv(f"{processed_path}/train-weighted.csv")
df_test.to_csv(f"{processed_path}/test-weighted.csv")